<a href="https://colab.research.google.com/github/Jieeee412/Lab10/blob/main/DS_COMPVAL_Lending_Fees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --upgrade pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 88.6 MB/s eta 0:00:00


In [83]:
import fitz  # PyMuPDF

pdf_path = "/content/Lending fees_7.pdf"
doc = fitz.open(pdf_path)

# Extract text by page
page_texts = {}

for i in range(len(doc)):
    text = doc.load_page(i).get_text()
    page_texts[f"page_{i+1}"] = text.strip()


In [9]:
import requests
import json
import time

# === CONFIGURATION ===

api_key = "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"
model_id = "deepseek/deepseek-r1-zero:free"
output_file = "Lending fees 7.json"
deepseek_outputs = {}

# === Prompt Template ===
#def build_prompt(page_text):
    #return (
        #"You are an intelligent document parser. Based on the following banking fee document content, "
        #"extract all product types and their associated fees in structured JSON format. "
        #"Exclude entries like 'no charge', 'none', or 'free'.\n\n"
        #f"{page_text}\n\n"
        #"Format your response like this:\n"
        #"{\n  \"credit card\": {\"Annual Fee\": \"$120\"},\n  \"everyday account\": {\"ATM Fee\": \"$2\"}\n}"
    #)

def build_prompt(page_text):
    return (
        f"""
You are a financial data extraction assistant.

From the following page text, extract all **lending-related fees** as structured JSON.

For each fee, include the following fields:
- `fee_name`
- `product_type`
- `amount` (e.g., 12.5 or null for "no charge")
- `unit` ("NZD", "%", or "variable")
- `charge_timing` (e.g., "monthly", "once-off", "event-based")
- `conditions` (optional explanation if provided)

Return a **JSON array**, grouped logically by `product_type`.

---
PAGE TEXT:
\"\"\"
{page_text}
\"\"\"
"""
    )

# === DeepSeek Query Function ===
def query_deepseek(prompt):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model_id,
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.ok:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f" Error {response.status_code}: {response.text}"

# === Run for All Pages ===
for page_name, page_text in page_texts.items():
    print(f" Processing {page_name}...")
    prompt = build_prompt(page_text)
    response = query_deepseek(prompt)
    deepseek_outputs[page_name] = response
    time.sleep(1)

# === Save Results ===
with open(output_file, "w") as f:
    json.dump(deepseek_outputs, f, indent=2)

print(f"\n All pages processed. Results saved to '{output_file}'")


 Processing page_1...

 All pages processed. Results saved to 'Lending fees 7.json'


In [10]:
import json

with open("Lending fees 7.json", "r") as f:
    all_pages = json.load(f)

# View one specific page (e.g., page 1)
print(all_pages['page_1'])


\boxed{
```json
[
    {
        "product_type": "Home Loan",
        "fees": [
            {
                "fee_name": "Application fee (owner-occupied)",
                "amount": null,
                "unit": "NZD",
                "charge_timing": "once-off",
                "conditions": "no charge"
            },
            {
                "fee_name": "Application fee (all other lending)",
                "amount": null,
                "unit": "NZD",
                "charge_timing": "once-off",
                "conditions": "no charge"
            },
            {
                "fee_name": "Loan Top Up fees",
                "amount": null,
                "unit": "NZD",
                "charge_timing": "once-off",
                "conditions": "no charge"
            },
            {
                "fee_name": "Tideover application/extension fee",
                "amount": null,
                "unit": "NZD",
                "charge_timing": "once-off",
                "

In [11]:
import json

with open("Lending fees 8.json", "r") as f:
    all_pages = json.load(f)

# View one specific page (e.g., page 1)
print(all_pages['page_1'])

\boxed{
[
    {
        "fee_name": "Discharge or Execution fee",
        "product_type": "secured loans",
        "amount": 100,
        "unit": "NZD",
        "charge_timing": "once-off",
        "conditions": "Charged for transactions related to security such as discharge, partial discharge, or release of security or securities. Submitted execution requests need to be made at least two days before the settlement date unless an urgent settlement is arranged."
    }
]}
